 import library

In [1]:
import pandas as pd
import numpy as np
import pydicom as dicom
import os
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

In [2]:
class_names = {'non-tumor': 0,
               'adenocarcinoma': 1,
               'hematopoietic ': 2,
               'squamous cell carcinoma ': 3,
               'melanoma': 4,
               'neuroendocrine carcinoma ': 5,
               'sarcoma ': 6,
               'urothelial carcinoma ': 7,
               }

excel_path = 'Mets_morphology_dataset_fin_V2.xlsx'
excel = pd.read_excel(excel_path)
excel['Histomorphology'] = excel['Histomorphology'].replace('Adenokarzinom', 'adenocarcinoma')
excel = excel.drop([215]).reset_index(drop = True)
excel['number'] = excel['number'].astype('str')

# class_names = {j:i for i,j in enumerate(excel['Histomorphology'].unique().tolist())}

n_classes = len(class_names)
class_labels = {excel['number'][i]:class_names[excel['Histomorphology'][i]]  for i in range(len(excel))}

In [ ]:
image_dir = 'Channel Images\Channel Images'
folders = os.listdir(image_dir)

if not(os.path.exists('concat_images')):
    os.mkdir('concat_images')

for i,p in enumerate(folders):
    image_subdir = os.path.join(image_dir,p)
    files = os.listdir(image_subdir)

    temp_df = pd.DataFrame({'col1': files})
    temp_df['number'] = temp_df['col1'].apply(lambda x: int(x.split('_')[1]))
    temp_df['name'] = temp_df['col1'].apply(lambda x: x.split('_')[0])

    for sub_df in temp_df.groupby('name'):
        sorted_files = sub_df[1].sort_values(by=['number']).reset_index(drop=True)['col1'].tolist()

        # get shape of a sample in sub directory
        shape = dicom.dcmread(os.path.join(image_subdir, sorted_files[0])).pixel_array.shape
        dcm_array1 = np.array([], dtype='uint16').reshape(shape[0],0)
        dcm_array2 = np.array([], dtype='uint16').reshape(shape[0],0)
        for j in range(0,len(sorted_files),2):
            # read dcm file
            dcm_sample = dicom.dcmread(os.path.join(image_subdir, sorted_files[j])).pixel_array
            dcm_array1 = np.concatenate((dcm_array1, dcm_sample), axis=1)
            dcm_sample = dicom.dcmread(os.path.join(image_subdir, sorted_files[j+1])).pixel_array
            dcm_array2 = np.concatenate((dcm_array2, dcm_sample), axis=1)
        # dcm_array1 = exposure.equalize_adapthist(dcm_array1)*255
        # dcm_array2 = exposure.equalize_adapthist(dcm_array2)*255

        label = class_labels[p]

        '''
        image_reg = register_mosaic(dcm_array1, dcm_array2)
        subtracted_array = np.subtract(image_reg[:,:,1], image_reg[:,:,0]) # CH3 minus CH2
        subtracted_array[subtracted_array < 0] = 0 # negative values set to zero
        dcm_stack = np.zeros((dcm_array1.shape[0], dcm_array1.shape[1], 3), dtype='uint16')
        dcm_stack[:, :, 0] = subtracted_array
        dcm_stack[:, :, 1] = image_reg[:,:,0]
        dcm_stack[:, :, 2] = image_reg[:,:,1]
        '''

        dcm_stack = np.zeros((dcm_array1.shape[0], dcm_array1.shape[1], 2), dtype='uint16')
        dcm_stack[:,:,0] = dcm_array1[:]
        dcm_stack[:,:,1] = dcm_array2[:]
        np.save('concat_images/{}_{}_label_{}_x_{}_y_{}_'.format(p, sub_df[0], label, dcm_stack.shape[0], dcm_stack.shape[1]), dcm_stack)

        # np.save('concat_images/{}_{}_1_label_{}_x_{}_y_{}_'.format(p, sub_df[0], label, dcm_array1.shape[0], dcm_array1.shape[1]), dcm_array1)
        # np.save('concat_images/{}_{}_2_label_{}_x_{}_y_{}_'.format(p, sub_df[0], label, dcm_array2.shape[0], dcm_array2.shape[1]), dcm_array2)

    print('folder {}/{}'.format(i+1,len(folders)))

# For non-Tumor class

In [ ]:
image_dir = 'negative controls/channel images'
folders = os.listdir(image_dir)

label = 0
if not(os.path.exists('concat_images')):
    os.mkdir('concat_images')

for i,p in enumerate(folders):
    image_subdir = os.path.join(image_dir,p)
    files = os.listdir(image_subdir)

    temp_df = pd.DataFrame({'col1': files})
    temp_df['number'] = temp_df['col1'].apply(lambda x: int(x.split('_')[1]))
    temp_df['name'] = temp_df['col1'].apply(lambda x: x.split('_')[0])

    for sub_df in temp_df.groupby('name'):
        sorted_files = sub_df[1].sort_values(by=['number']).reset_index(drop=True)['col1'].tolist()

        # get shape of a sample in sub directory
        shape = dicom.dcmread(os.path.join(image_subdir, sorted_files[0])).pixel_array.shape
        dcm_array1 = np.array([], dtype='uint16').reshape(shape[0],0)
        dcm_array2 = np.array([], dtype='uint16').reshape(shape[0],0)
        for j in range(0,len(sorted_files),2):
            # read dcm file
            dcm_sample = dicom.dcmread(os.path.join(image_subdir, sorted_files[j])).pixel_array
            dcm_array1 = np.concatenate((dcm_array1, dcm_sample), axis=1)
            dcm_sample = dicom.dcmread(os.path.join(image_subdir, sorted_files[j+1])).pixel_array
            dcm_array2 = np.concatenate((dcm_array2, dcm_sample), axis=1)
        # dcm_array1 = exposure.equalize_adapthist(dcm_array1)*255
        # dcm_array2 = exposure.equalize_adapthist(dcm_array2)*255
        '''
        image_reg = register_mosaic(dcm_array1, dcm_array2)

        subtracted_array = np.subtract(image_reg[:,:,1], image_reg[:,:,0]) # CH3 minus CH2
        subtracted_array[subtracted_array < 0] = 0 # negative values set to zero

        dcm_stack = np.zeros((dcm_array1.shape[0], dcm_array1.shape[1], 3), dtype='uint16')
        dcm_stack[:, :, 0] = subtracted_array
        dcm_stack[:, :, 1] = image_reg[:,:,0]
        dcm_stack[:, :, 2] = image_reg[:,:,1]
        np.save('concat_images/{}_{}_label_{}_x_{}_y_{}_'.format(p, sub_df[0], 0, dcm_stack.shape[0], dcm_stack.shape[1]), dcm_stack)
        '''

        dcm_stack = np.zeros((dcm_array1.shape[0], dcm_array1.shape[1], 2), dtype='uint16')
        dcm_stack[:,:,0] = dcm_array1[:]
        dcm_stack[:,:,1] = dcm_array2[:]
        np.save('concat_images/{}_{}_label_{}_x_{}_y_{}_'.format(p, sub_df[0], label, dcm_stack.shape[0], dcm_stack.shape[1]), dcm_stack)



        # print(np.max(dcm_array1), np.max(dcm_array2))
    print('folder {}/{}'.format(i+1,len(folders)))